# Projeto Aplicado III - Construindo um sistema de recomendação

O projeto tem como objetivo **desenvolver um sistema de recomendação de filmes** utilizando o *5000 Movie Dataset*, disponível no Kaggle, que reúne informações detalhadas sobre cerca de 5.000 produções do The Movie Database (TMDb). Esse conjunto de dados inclui variáveis como **orçamento, gêneros, popularidade, empresas produtoras, países de produção, elenco e equipe técnica**, permitindo a aplicação de técnicas de aprendizado de máquina para sugerir conteúdos mais relevantes aos usuários.

A iniciativa busca não apenas aprimorar competências práticas em **ciência de dados e mineração de dados**, mas também contribuir para os **Objetivos de Desenvolvimento Sustentável (ODS)** da ONU, como o **ODS 9 (Inovação e Infraestrutura)**, o **ODS 4 (Educação de Qualidade)** e o **ODS 10 (Redução das Desigualdades)**. Assim, o sistema pretende oferecer recomendações personalizadas que promovam maior **diversidade cultural e inclusão digital**


Para a análise e desenvolvimento deste projeto, será necessário utilizar um conjunto de bibliotecas do ecossistema Python, cada uma com um papel específico no fluxo de trabalho de **análise exploratória** e **construção do sistema de recomendação**.

- **pandas**: essencial para manipulação e análise de dados tabulares, permitindo leitura, limpeza e transformação dos datasets `tmdb_5000_movies` e `tmdb_5000_credits`.  
- **numpy**: fornece suporte para operações matemáticas e vetorização, aumentando a eficiência no processamento de dados.  
- **matplotlib**: utilizada para criar visualizações básicas, como gráficos de barras, dispersão e histogramas.  
- **seaborn**: complementa o matplotlib oferecendo visualizações estatísticas mais sofisticadas e com estética aprimorada.  
- **scikit-learn**: importante para o pré-processamento dos dados, cálculo de métricas de avaliação e implementação de algoritmos de recomendação baseados em aprendizado de máquina.  
- **scipy**: será usada para cálculos matemáticos 
- **surprise (scikit-surprise)**: biblioteca especializada em sistemas de recomendação, especialmente nos modelos colaborativos como **SVD** e **KNNBasic**, permitindo comparar técnicas.  
- **lightfm**: possibilita a construção de sistemas de recomendação híbridos, combinando informações de conteúdo e interações de usuários.  
- **tensorflow / pytorch**: úteis caso seja necessário evoluir para modelos de recomendação mais avançados baseados em **deep learning**.


Abaixo, será realizada a importação das bibliotecas necessárias para a **análise exploratória** e a **construção do sistema de recomendação**.

Antes de iniciar a análise, é necessário garantir que todas as bibliotecas utilizadas neste projeto estejam instaladas no ambiente.  
Para isso, basta executar o comando abaixo no terminal ou em uma célula do Jupyter Notebook (prefixado com `!`):

```bash
pip install -r requirements.txt


In [ ]:
# Bibliotecas para análise de dados
import pandas as pd
import numpy as np
import json
import ast

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para sistemas de recomendação
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import scipy
from surprise import Dataset, Reader, SVD, KNNBasic
from lightfm import LightFM

# Deep Learning (opcional, dependendo da abordagem)
import tensorflow as tf
import torch


In [ ]:
# Leitura dos datasets
movies_df = pd.read_csv("datasets/tmdb_5000_movies.csv")
credits_df = pd.read_csv("datasets/tmdb_5000_credits.csv")

# Mostrando as primeiras linhas dos datasets
print("📌 Movies Dataset:")
display(movies_df.head())

print("\n📌 Credits Dataset:")
display(credits_df.head())

# Informações básicas dos datasets
print("\n🔎 Informações do Movies Dataset:")
print(movies_df.info())

print("\n🔎 Informações do Credits Dataset:")
print(credits_df.info())


Nesta etapa, será realizada a **seleção das colunas relevantes** para a construção do sistema de recomendação.  

O objetivo é reduzir o dataframe apenas às variáveis que realmente serão utilizadas no modelo, garantindo maior **eficiência no processamento**, evitando redundâncias e mantendo o foco nos atributos mais informativos para a geração das recomendações.

Além disso, também serão tratados os valores nulos para diminuir o ruído presente no dataset.

Também será feito um **merge com o dataframe de créditos**, permitindo integrar informações sobre elenco e equipe técnica ao conjunto de dados principal, enriquecendo assim a base para a construção do sistema de recomendação.


In [ ]:
# Movies Dataset

# Filtrando somente linhas em que o filme está com status 'Released'
# Utilizamos o reset_index(drop=True) para resetar os índices do DataFrame após o filtro
movies_df = movies_df[movies_df['status'] == 'Released'].reset_index(drop=True)
# Agora, filtraremos apenas as colunas que nos interessam para a análise
movies_df = movies_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'release_date', 'overview', 'runtime', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']]
# Tratando linhas com valores nulos
movies_df = movies_df.dropna().reset_index(drop=True)
# Convertendo a coluna 'release_date' para o tipo datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

# Credits Dataset
# Selecionando apenas as colunas que nos interessam
credits_df = credits_df[['movie_id', 'cast', 'crew']]
# Tratando linhas com valores nulos
credits_df = credits_df.dropna().reset_index(drop=True)
# Renomeando a coluna 'movie_id' para 'id' para facilitar o merge
credits_df = credits_df.rename(columns={'movie_id': 'id'})

# Merge dos datasets
# Realizando o merge dos datasets 'movies_df' e 'credits_df' com base na coluna 'id'
movies_df = movies_df.merge(credits_df, on='id')
# Mostrando as primeiras linhas do dataset final
print("\n📌 Dataset Final após Merge:")
display(movies_df)


Neste trecho de código é realizado o **tratamento das colunas no formato JSON** presentes nos datasets.  
Foram criadas funções auxiliares para converter as colunas que armazenam listas de dicionários em **listas de valores extraídos**, de forma a facilitar a manipulação e análise.  

- A função `parse_json_column` é responsável por percorrer colunas no formato JSON e **extrair os valores de um campo específico** (como `name`, `iso_3166_1` ou `iso_639_1`).  
- A função `extract_director` percorre a coluna `crew` e **identifica o diretor principal** de cada filme, criando uma nova coluna `director`.  

Após o processamento, as colunas originais em JSON são transformadas em listas de valores mais simples, e a coluna `crew` foi descartada, visto que sua informação relevante (o diretor) já foi extraída.  
Esse tratamento torna o dataframe mais **limpo e estruturado**, possibilitando seu uso na análise exploratória e na construção do sistema de recomendação.

In [ ]:
# Funções para processar as colunas JSON
def parse_json_column(df, column_name, key='name'):
    def extract_names(json_str):
        if isinstance(json_str, str):
            try:
                list_of_dicts = ast.literal_eval(json_str)
                if isinstance(list_of_dicts, list):
                    return [d[key] for d in list_of_dicts if key in d]
            except (ValueError, SyntaxError):
                pass
        return []
    df[column_name] = df[column_name].apply(extract_names)
    return df

# Método para extrair o diretor do elenco
def extract_director(crew_json):
    if isinstance(crew_json, str):
        try:
            crew_list = ast.literal_eval(crew_json)
            for member in crew_list:
                if member.get('job') == 'Director':
                    return member.get('name')
        except (ValueError, SyntaxError):
            pass
    return np.nan

# Aplicando as funções para explodir as colunas JSON
movies_df = parse_json_column(movies_df, 'genres')
movies_df = parse_json_column(movies_df, 'keywords')
movies_df = parse_json_column(movies_df, 'production_companies')
movies_df = parse_json_column(movies_df, 'production_countries', key='iso_3166_1') 
movies_df = parse_json_column(movies_df, 'spoken_languages', key='iso_639_1')
movies_df = parse_json_column(movies_df, 'cast', key='name')
movies_df['director'] = movies_df['crew'].apply(extract_director)

# Removendo a coluna 'crew' original, pois já extraímos o diretor
movies_df = movies_df.drop(columns=['crew'])

# Mostrando as primeiras linhas do dataset após o tratamento das colunas JSON
print("\n📌 Dataset Final após tratamento de JSON:")
display(movies_df.head())